# File Suggestions

The next phase of knowledge graph is to suggest files to use for import, based on the established user goal.

## Agent

- An agent that suggests files to use for import, based on the established user goal.
- Input: `approved_user_goal`, a dictionary pairing a kind of graph with a description of the purpose of the graph.
- Output: `approved_file_suggestions`, a list of files that have been approved for import.
- Tools: `get_approved_user_goal`, `list_import_files`, `sample_file`, `set_suggested_files`, `approve_suggested_files`

## Workflow

1. The context is initialized with an `approved_user_goal` (which will get retrieved using a tool)
2. The agent analyzes the available files, evaluating them for relevance to the established user goal.
3. The agent suggests a list of files to import.
4. The user approves the file suggestions.
5. The file suggestions are saved in the context state as `approved_file_suggestions`.


## Setup

The usual import of needed libraries, loading of environment variables, and connection to Neo4j.

In [ ]:
# Import necessary libraries
import os
from pathlib import Path

from itertools import islice

from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For OpenAI support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools import ToolContext
from google.genai import types # For creating message Content/Parts

# For type hints
from typing import Dict, Any, List

# Convenience libraries for working with Neo4j inside of Google ADK
from neo4j_for_adk import graphdb, tool_success, tool_error

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.CRITICAL)

print("Libraries imported.")

In [ ]:
# --- Define Model Constants for easier use ---
MODEL_GPT_4O = "openai/gpt-4o"

llm = LiteLlm(model=MODEL_GPT_4O)

# Test LLM with a direct call
print(llm.llm_client.completion(model=llm.model, messages=[{"role": "user", "content": "Are you ready?"}], tools=[]))

print("\nEnvironment configured.")

In [ ]:
# Check connection to Neo4j by sending a query

neo4j_is_ready = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")

print(neo4j_is_ready)

## Define the File Suggestion Agent

### Agent Instructions

In [ ]:
# First, define the instruction to describe what the agent should do
file_suggestion_agent_instruction = """
        You are a constructive critic AI reviewing a list of files. Your goal is to suggest relevant files.

        **Task:**
        Review the file list for relevance to the kind of graph and description specified in: '{{approved_user_goal}}'. 

        For any file that you're not sure about, use the 'sample_file' tool to get 
        a better understanding of the file contents. 
        You do not need to sample every file. Assume markdown files in the same directory have similar features.
        Sample only a few markdown files, and if they are relevant suggest every markdown file in the directory.

        Think carefully, repeating these steps until finished:
        1. list available files
        2. evaluate the relevance of each file, then set the list of suggested files using the 'set_suggested_files' tool
        3. ask the user to approve the set of suggested files
        4. If the user has feedback, go back to step 1 with that feedback in mind
        5. If approved, use the 'approve_suggested_files' tool to record the approval
        """


### Tool Definitions

In [ ]:
# import tools defined in previous notebook
from tools import get_approved_user_goal

In [ ]:
#  Tool: List Import Files

ALL_AVAILABLE_FILES = "all_available_files"

def list_import_files(tool_context:ToolContext) -> dict:
    f"""Lists files available in the configured Neo4j import directory
    that are ready for import by Neo4j.

    Saves the list to {ALL_AVAILABLE_FILES} in state.

    Returns:
        dict: A dictionary containing metadata about the content.
                Includes a 'status' key ('success' or 'error').
                If 'success', includes a {ALL_AVAILABLE_FILES} key with list of file names.
                If 'error', includes an 'error_message' key.
                The 'error_message' may have instructions about how to handle the error.
    """
    import_dir_result = graphdb.get_import_directory() # use the helper available in Neo4jForADK
    if import_dir_result["status"] == "error": return import_dir_result
    import_dir = Path(import_dir_result["neo4j_import_dir"])

    file_names = [str(x.relative_to(import_dir)) 
                 for x in import_dir.rglob("*") 
                 if x.is_file()]

    tool_context.state[ALL_AVAILABLE_FILES] = file_names

    return tool_success(ALL_AVAILABLE_FILES, file_names)


In [ ]:
# Tool: Sample File
def sample_file(file_path: str, tool_context: ToolContext) -> dict:
    """Samples a file by reading its content as text.
    
    Treats any file as text and reads up to a maximum of 100 lines.
    
    Args:
      file_path: file to sample, relative to the import directory
      tool_context: ToolContext object
      
    Returns:
        dict: A dictionary containing metadata about the content,
              along with a sampling of the file.
              Includes a 'status' key ('success' or 'error').
              If 'success', includes a 'content' key with textual file content.
              If 'error', includes an 'error_message' key.
    """
    import_dir_result = graphdb.get_import_directory() # use the helper available in Neo4jForADK
    if import_dir_result["status"] == "error": return import_dir_result
    import_dir = Path(import_dir_result["neo4j_import_dir"])

    full_path_to_file = import_dir / file_path
    
    if not full_path_to_file.exists():
        return tool_error(f"File does not exist in import directory: {file_path}")
    
    try:
        # Treat all files as text
        with open(full_path_to_file, 'r', encoding='utf-8') as file:
            # Read up to 100 lines
            lines = list(islice(file, 100))
            content = ''.join(lines)
            return tool_success("content", content)
    
    except Exception as e:
        return tool_error(f"Error reading or processing file {file_path}: {e}")

In [ ]:
SUGGESTED_FILES = "suggested_files"

def set_suggested_files(suggest_files:List[str], tool_context:ToolContext) -> Dict[str, Any]:
    """Set the files to be used for data import.
    """
    tool_context.state[SUGGESTED_FILES] = suggest_files
    return tool_success(SUGGESTED_FILES, suggest_files)

In [ ]:
APPROVED_FILES = "approved_files"

def approve_suggested_files(tool_context:ToolContext) -> Dict[str, Any]:
    f"""Approves the {SUGGESTED_FILES} in state for further processing as {APPROVED_FILES}."""
    
    if SUGGESTED_FILES not in tool_context.state:
        return tool_error("Current files have not been set. Take no action other than to inform user.")

    tool_context.state[APPROVED_FILES] = tool_context.state[SUGGESTED_FILES]

In [ ]:
# List of tools for the file suggestion agent
file_suggestion_agent_tools = [get_approved_user_goal, list_import_files, sample_file, set_suggested_files, approve_suggested_files]

### Construct the Agent

In [ ]:
# Finally, construct the agent

file_suggestion_agent = Agent(
    name="file_suggestion_agent_v1",
    model=llm, # defined earlier in a variable
    description="Helps the user select files to import.",
    instruction=file_suggestion_agent_instruction,
    tools=file_suggestion_agent_tools,
)

print(f"Agent '{file_suggestion_agent.name}' created.")

---

## Interact with the Agent



In [ ]:
# Define an Agent Caller Utility
# This will provide a simple "call" interface and access to the session

from helpers import make_agent_caller

file_suggestion_caller = make_agent_caller(file_suggestion_agent, {
    "approved_user_goal": {
        "kind_of_graph": "movie graph", # TODO: change to a BOM graph
        "description": "Movies, actors and acted-in relationships for study of co-acting group behaviors."
    }   
})


In [ ]:
# Run the Initial Conversation
await file_suggestion_caller.call("What files can we use for import?", True)

print("Suggested files: ", file_suggestion_caller.session.state[SUGGESTED_FILES])


In [ ]:
# Agree with the file suggestions
await file_suggestion_caller.call("Yes, let's do it!", True)

print("Approved files: ", file_suggestion_caller.session.state[APPROVED_FILES])



---

Congratulations\! You've created a basic human-in-the-loop interaction, with a structured result.


---
## Bonus, An Interactive Conversation

Now, let's make this interactive so you can ask your own questions! Run the cell below. It will prompt you to enter your queries directly.

In [ ]:
async def run_interactive_conversation():
    while True:
        user_query = input("Ask me something (or type 'exit' to quit): ")
        if user_query.lower() == 'exit':
            break
        response = await file_suggestion_caller.call(user_query, True)
        print(f"Response: {response}")

# Execute the interactive conversation
await run_interactive_conversation()